### Imports

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from feature_baggingV2 import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import datetime
from sklearn.preprocessing import MinMaxScaler

import os, sys, gc, time, warnings, pickle, psutil, random

### Función para escalar/desescalar y métrica

In [2]:
# Function to center, scale, and return a series
def minmax_scale_group(group):
    median = group.median()
    centered_values = group - median
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(centered_values.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    medians[group.name] = median  # Store the median for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_minmax_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    median = medians[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values + median
    return pd.Series(original_values, index=group.index, name=group_name)



### Archivos

In [3]:
#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'
df_sell_in = pd.read_csv(DATOS_DIR+'sell-in.txt', sep='\t')
df_predecir = pd.read_csv(DATOS_DIR+'productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv(DATOS_DIR+'tb_stocks.txt', sep='\t')
df_tb_productos = pd.read_csv(DATOS_DIR+'tb_productos_descripcion.txt', sep='\t')

Si quiero predecir algun mes en especifico y filtrar los productos con más de 3 meses de venta

In [4]:
#df_sell_in = df_sell_in[df_sell_in['periodo']<201808]
#df_sell_in['tn_2'] = np.where(df_sell_in['periodo'].isin([201805, 201804]), np.nan, df_sell_in['tn_2']) 
#df_sell_in = df_sell_in[df_sell_in['periodo']<201806]
#sales_counts = df_sell_in.groupby('product_id')['periodo'].count() # Contar el número de meses de ventas por product_id
#products_with_more_than_3_months = sales_counts[sales_counts > 3].index # Filtrar los productos con más de 3 meses de ventas
#df_sell_in = df_sell_in[df_sell_in['product_id'].isin(products_with_more_than_3_months)] # Filtrar el DataFrame original para incluir solo estos productos

### Preprocesamiento

In [5]:
df_tb_stocks['periodo'] = pd.to_datetime(df_tb_stocks['periodo'], format='%Y%m')
df_tb_stocks['periodo'] = df_tb_stocks['periodo'] - pd.DateOffset(months=1) #mes diferente en stock
df_tb_stocks['product_id'] = df_tb_stocks['product_id'].astype(int)
df_tb_stocks['stock_final'] = df_tb_stocks['stock_final'].astype(float)
df_tb_productos['product_id'] = df_tb_productos['product_id'].astype(int)
df_tb_productos['sku_size'] = df_tb_productos['sku_size'].astype(int)
df_sell_in['periodo'] = pd.to_datetime(df_sell_in['periodo'], format='%Y%m')
df_sell_in['product_id'] = df_sell_in['product_id'].astype(int)
df_sell_in['customer_id'] = df_sell_in['customer_id'].astype(int)
df_sell_in['cust_request_qty'] = df_sell_in['cust_request_qty'].astype(int)
df_sell_in['cust_request_tn'] = df_sell_in['cust_request_tn'].astype(float)
df_sell_in['tn'] = df_sell_in['tn'].astype(float)
df_sell_in['plan_precios_cuidados'] = df_sell_in['plan_precios_cuidados'].astype(bool)

### Consolidar Datos

In [6]:
df_sell_in['tn_2'] = df_sell_in['tn'].shift(-2) #OBTENGO TN 2 MESES ADELANTE (CLASE FINAL)

In [7]:
# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')

#Convertir 'periodo' a formato de fecha y Calcular el trimestre desde 'periodo'
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
df_final["trimestre"] = df_final.periodo.dt.quarter

#Establecer 'fecha' como índice y convertir a período mensual
df_final.set_index('fecha', inplace=True)
df_final.index = df_final.index.to_period('M')

In [8]:
df_final = df_final[df_final['product_id'].isin(df_predecir['product_id'])] #Filtrar solo los productos a predecir

### Clase de LightGBM

In [9]:
df_final['diff_tn_tn2'] =  df_final['tn_2'] - df_final['tn'] #NUEVA CLASE
#### Agrupar y escalar
scalers = {}
medians = {}
df_final['diff_tn_tn2_escalada'] = df_final.groupby('product_id')['diff_tn_tn2'].transform(minmax_scale_group) 

## FE

### Lags

In [10]:
# Create lag variables for 'cust_request_qty', 'cust_request_tn', and 'tn'
n_lags = 36
for lag in range(1, n_lags + 1):
    df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
    df_final[f'cust_request_qty_lag_{lag}_escalada'] = df_final.groupby('product_id')[f'cust_request_qty_lag_{lag}'].transform(minmax_scale_group) 
    
    df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
    df_final[f'cust_request_tn_lag_{lag}_escalada'] = df_final.groupby('product_id')[f'cust_request_tn_lag_{lag}'].transform(minmax_scale_group) 
   
    df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
    df_final[f'stock_final_lag_{lag}_escalada'] = df_final.groupby('product_id')[f'stock_final_lag_{lag}'].transform(minmax_scale_group) 
   
    df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)
    df_final[f'tn_lag_{lag}_escalada'] = df_final.groupby('product_id')[f'tn_lag_{lag}'].transform(minmax_scale_group) 
   

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2562115973.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2562115973.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'cust_request_qty_lag_{lag}_escalada'] = df_final.groupby('product_id')[f'cust_request_qty_lag_{lag}'].transform(minmax_scale_group)
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2562115973.py:7: PerformanceWarning:

In [11]:
#DELTA DE LAGS 2 ANTERIORES

for lag in range(1, n_lags-1):
    df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
        df_final[f'cust_request_tn_lag_{lag+2}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
        df_final[f'stock_final_lag_{lag+2}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+2}'] = (
        df_final[f'tn_lag_{lag+2}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\260571944.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\260571944.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\260571944.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

In [12]:
#DELTA DE LAGS 1 ANTERIOR

for lag in range(1, n_lags):
    df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
        df_final[f'cust_request_tn_lag_{lag+1}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
        df_final[f'stock_final_lag_{lag+1}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+1}'] = (
        df_final[f'tn_lag_{lag+1}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\1785296116.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\1785296116.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\1785296116.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [13]:
#Mes actual / (lag 2 + lag 3) 

# Calcular el ratio del valor actual con respecto a la suma de los dos lags anteriores
for lag in range(3, n_lags + 1):
    df_final[f'ratio_cust_request_tn_{lag}'] = (
        df_final[f'cust_request_tn_lag_{lag}'] / (
            df_final[f'cust_request_tn_lag_{lag-1}'] + df_final[f'cust_request_tn_lag_{lag-2}']
        )
    )
    df_final[f'ratio_stock_final_{lag}'] = (
        df_final[f'stock_final_lag_{lag}'] / (
            df_final[f'stock_final_lag_{lag-1}'] + df_final[f'stock_final_lag_{lag-2}']
        )
    )
    df_final[f'ratio_tn_{lag}'] = (
        df_final[f'tn_lag_{lag}'] / (
            df_final[f'tn_lag_{lag-1}'] + df_final[f'tn_lag_{lag-2}']
        )
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2974401302.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_cust_request_tn_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2974401302.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_stock_final_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\2974401302.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [14]:
df_final.reset_index(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3586226101.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.reset_index(inplace=True)


### Medias Móviles

In [15]:
#MEDIAS MOVILES
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
    df_final[f'rolling_mean_tn_{window}_escalada'] = df_final.groupby('product_id')[f'rolling_mean_tn_{window}'].transform(minmax_scale_group) 

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3180062501.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3180062501.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}_escalada'] = df_final.groupby('product_id')[f'rolling_mean_tn_{window}'].transform(minmax_scale_group)
C:\Users\Admin\AppData\Local\Temp

### FE

In [16]:
# Datetime features
df_final['year'] = df_final['periodo'].dt.year
df_final['month'] = df_final['periodo'].dt.month
df_final['quarter'] = df_final.periodo.dt.quarter

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3544365912.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['year'] = df_final['periodo'].dt.year
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3544365912.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['month'] = df_final['periodo'].dt.month
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3544365912.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

In [17]:
#Variables Dummies si es el max o el min de cierta cantidad de meses
months = [3, 6, 9, 12]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for i in months:
    df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
    df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
    # Crear las dummies
    df_final[f'dummy_max_{i}m'] = np.where(df_final['tn'] == df_final[f'max_{i}m'], 1, 0)
    df_final[f'dummy_min_{i}m'] = np.where(df_final['tn'] == df_final[f'min_{i}m'], 1, 0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3137070357.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3137070357.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3137070357.py:9: PerformanceW

In [18]:
#Calcular ventas por trimestre
df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')

df_final['tn_trimestre_escalada'] = df_final.groupby('product_id')['tn_trimestre'].transform(minmax_scale_group) 
#Calcular ventas por trimestre por cliente
df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')

df_final['ttn_trimestre_customer_escalada'] = df_final.groupby('product_id')['tn_trimestre_customer'].transform(minmax_scale_group) 

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3317141678.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3317141678.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre_escalada'] = df_final.groupby('product_id')['tn_trimestre'].transform(minmax_scale_group)
C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\3317141678.py:6: PerformanceWarning: DataFrame is hi

In [19]:
df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')

C:\Users\Admin\AppData\Local\Temp\ipykernel_7620\4104376824.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')


### FE variables externas

In [20]:
# Leer el archivo exportado
df_exported = pd.read_excel(DATOS_DIR+'23variables_externas.xlsx')

# Asegúrate de que las columnas de fecha estén en el formato datetime
df_exported['fecha'] = pd.to_datetime(df_exported['fecha'])

# Unir los DataFrames por la columna de fecha
df_merged = pd.merge(df_final, df_exported, on='fecha', how='left')


In [21]:
df_final['plan_precios_cuidados'] =df_final['plan_precios_cuidados'].astype(bool)
#df_final['dias_fin_trimestre'] = df_final['dias_fin_trimestre'].dt.days.astype(int) #TARDA!
df_final = df_final.drop(columns=['fecha'])

## Inicio Train

In [22]:
# #Cambiar las variables categoricas y hacer one-hot encoding

df_final["cat1"] = df_final["cat1"].astype("category")
df_final["cat2"] = df_final["cat2"].astype("category")
df_final["cat3"] = df_final["cat3"].astype("category")
df_final["brand"] = df_final["brand"].astype("category")
df_final["descripcion"] = df_final["descripcion"].astype("category")

# # Encode categorical variables explicitly. One-hot encoding
cat1_dummies = pd.get_dummies(df_final['cat1'], prefix='cat1', drop_first=True)
cat2_dummies = pd.get_dummies(df_final['cat2'], prefix='cat2', drop_first=True)
cat3_dummies = pd.get_dummies(df_final['cat3'], prefix='cat3', drop_first=True)
brand_dummies = pd.get_dummies(df_final['brand'], prefix='brand', drop_first=True)
descripcion_dummies = pd.get_dummies(df_final['descripcion'], prefix='descripcion', drop_first=True)

# # Concatenate the dummy variables to the DataFrame and drop the original categorical columns
df_final= pd.concat([df_final, cat1_dummies, cat2_dummies, cat3_dummies, brand_dummies, descripcion_dummies], axis=1)
df_final.drop(columns=['cat1', 'cat2', 'cat3', 'brand'], inplace=True)

df_final.set_index('periodo', inplace=True)
df_final.index = df_final.index.to_period('M')
df_final.sort_index(inplace=True)


In [23]:

df_final.to_parquet(DATOS_DIR+'/FE_02_dataset.parquet', engine='pyarrow')  